In [ ]:
!pip install --upgrade cohere tiktoken openai wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.9/220.9 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00


In [ ]:
import os
import openai
import wandb


In [ ]:
run = wandb.init(project='GPT-4 in Python')
prediction_table = wandb.Table(columns=["prompt", "prompt tokens", "completion", "completion tokens", "model", "total tokens"])

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
gpt_prompt = input ("What prompt do you want to use?")
print(gpt_prompt)


What prompt do you want to use?h
h


In [ ]:
from openai import OpenAI

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key="",
)

'''
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "hello. how are you?",
        }
    ],
    model="gpt-4-0314",
)
'''
content = '''
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.
The question is...
context: Figure: Yasuni National Park.\nThe Amazon rain forest in South America is the largest rain forest ecosystem in the world. Ecuador's Yasuni National Park, which is in the Amazon rain forest, has many different species of plants, birds, and mammals. Figure:  'A large green forest with lots of trees.'  Question: Which statement describes the Yasuni National Park ecosystem? Optioons: 0. It has mostly small plants. 1. It has many different types of organisms. 2. It has soil that is rich in nutrients. \nAnswer:
'''

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": content,
        }
    ],
    model="gpt-3.5-turbo",
)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
main_path = "../drive/MyDrive/ScienceQA_DATA/"
main_path = "drive/MyDrive/ScienceQA_DATA/"

Mounted at /content/drive


In [ ]:
import json
def read_json(json_file_path):
  # Open the file in read mode
  with open(json_file_path, 'r') as file:
    # Load the JSON data from the file
    data = json.load(file)
  return data

latest_path = main_path + 'all_combined_result.json'
all_data = read_json(latest_path)

#for pid, content in fail_data.items():
#  prompt = f"context: {content["hint"]}"



In [ ]:
import json

def filter_json(input_file, output_file, target_subject):
    # Read the input JSON file
    with open(input_file, 'r') as f:
        data = json.load(f)

    total_samples = 0
    llava_correct = 0
    cot_correct = 0
    llama_correct = 0
    llama_zero_correct = 0

    # filtered_data = {
    #     entry for entry in data.items()  # Assuming entries are inside a dictionary
    #     # entry for entry in data.values()  # Assuming entries are inside a dictionary
    #     # if entry.get("subject", "") == target_subject
    #     # if entry.get("cot_answer") != str(entry.get("answer")) and entry.get("llava_answer") != str(entry.get("answer"))
    #     if entry.get("llava_finetuned_answer") != str(entry.get("answer"))
    # }
    filtered_data = {}
    for entry in data.keys():
        if data[entry].get("llava_finetuned_answer") != str(data[entry].get("answer")) and data[entry].get("cot_answer") != str(data[entry].get("answer")):
            filtered_data[entry] = data[entry]
    # for e in filtered_data:
    #     total_samples += 1
    #     answer = str(e.get("answer"))
    #     if e.get("llava_finetuned_answer") == answer:
    #         llava_correct += 1
    #     if e.get("cot_answer") == answer:
    #         cot_correct += 1
    #     if e.get("llama_output") == answer:
    #         llama_correct += 1
    #     if e.get("llama_zero_shot_output") == answer:
    #         llama_zero_correct += 1

    print(f"finetuned llava: {len(filtered_data)}")

    # Write the filtered data to a new JSON file
    with open(output_file, 'w') as f:
        json.dump(filtered_data, f, indent=2)

# Example usage:
input_json_file = main_path + 'all_combined_result.json'
output_json_file = 'fail_both_cot_and_llava.json'
target_subject = 'natural science'
# target_subject = 'language science'
# target_subject = 'social science'

filter_json(input_json_file, output_json_file, target_subject)

finetuned llava: 60


In [ ]:
path = 'fail_both_cot_and_llava.json'
fail_data = read_json(path)

In [ ]:
header = '''
Imagine three different experts are answering this question.
All experts will write down 1 step of their thinking,
then share it with the group.
Then all experts will go on to the next step, etc.
If any expert realises they're wrong at any point then they leave.

'''
#Show their discussion for all steps and conclude the correct option at the end.
prompts = []
for pid, content in fail_data.items():
  choices = content['choices']
  #print(content['hint'])
  #prompt = f"context: {content['hint'] figure: {content['accurate_image_captions']} question: {content['question']} options: "
  prompt = f"figure: {content['accurate_image_captions']} question: {content['question']} options: "

  for i in range(0,len(choices)):
    this_choice = f"{i}. {choices[i]} "
    prompt += this_choice

  #prompt = header + prompt +"\n\n Correct Option: "
  prompt = prompt +" Correct Option: "
  print(pid)
  #print(prompt)
  prompts.append(prompt)
  '''
  chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt,
        }
    ],
    model="gpt-3.5-turbo",
  )

  for c in chat_completion.choices:
    print(c.message.content)
    fail_data[pid]['tot_prompting'] = c.message.content
  '''
  #break



In [ ]:
for i in range(len(prompts)):
  if '\n' in prompts[i]:
    print(i)

10
31
38


In [ ]:
prompts[38]

'figure: In the image, there are two rectangular boxes with purple dots on them. Each box has a different number of dots, with one box having more dots than the other. Additionally, there is an arrow pointing towards the box with more dots, indicating that it contains more dots than the other box. The boxes are positioned side-by-side, making it easier to compare the number of dots in each box. question: Complete the text to describe the diagram.\nSolute particles moved in both directions across the permeable membrane. But more solute particles moved across the membrane (). When there was an equal concentration on both sides, the particles reached equilibrium. options: 0. to the left than to the right 1. to the right than to the left  Correct Option: '

In [ ]:
prompts[38] = prompts[38].replace('\n', ' ')

In [ ]:
# Open a file in write mode ('w')
with open('prompts_for_tot.txt', 'w') as file:
    # Iterate through the list and write each element to the file
    for element in prompts:
        file.write(element + '\n')

In [ ]:
# Specify the file path
file_path = main_path+'fail_case_gpt35_tot_prompting_result.json'

# Save the dictionary to a JSON file
with open(file_path, 'w') as json_file:
    json.dump(fail_data, json_file)

In [ ]:
print(chat_completion)

ChatCompletion(id='chatcmpl-8Q4d4RkrY3lXoSqin1x6gmNj1ZY1N', choices=[Choice(finish_reason='stop', index=0, message=ChatCompletionMessage(content="Hello! I'm an AI, so I don't have feelings, but I'm here to assist you. How can I help you today?", role='assistant', function_call=None, tool_calls=None))], created=1701225014, model='gpt-3.5-turbo-0613', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=29, prompt_tokens=13, total_tokens=42))


In [ ]:
for c in chat_completion.choices:
  print(type(c))
  print(c.message.content)

<class 'openai.types.chat.chat_completion.Choice'>
Expert 1: Step 1: Based on the given context of Yasuni National Park being in the Amazon rainforest, it is safe to assume that the ecosystem in the park would have a high diversity of organisms.
Expert 2: Step 1: Looking at the provided figure of a large green forest with lots of trees, it suggests that the Yasuni National Park ecosystem has a dense vegetation cover.
Expert 3: Step 1: Considering the context of the Amazon rainforest, it is known for its fertile soil due to high decomposition rates and nutrient cycling. Therefore, it can be inferred that the Yasuni National Park ecosystem has soil that is rich in nutrients.

Expert 1: Step 2: Given the natural diversity of the Amazon rainforest, it is reasonable to conclude that Yasuni National Park would have many different types of organisms, including plants, birds, and mammals.
Expert 2: Step 2: Due to the abundance of trees and dense vegetation observed in the figure, it supports t

In [ ]:
!pip install tree-of-thoughts-llm

In [ ]:
!git clone https://github.com/princeton-nlp/tree-of-thought-llm
%cd tree-of-thought-llm
!pip install -r requirements.txt
!pip install -e .  # install `tot` package

Cloning into 'tree-of-thought-llm'...
remote: Enumerating objects: 224, done.
remote: Counting objects: 100% (79/79), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 224 (delta 51), reused 50 (delta 45), pack-reused 145
Receiving objects: 100% (224/224), 2.17 MiB | 11.10 MiB/s, done.
Resolving deltas: 100% (100/100), done.
/content/tree-of-thought-llm
Obtaining file:///content/tree-of-thought-llm
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for tree-of-thoughts-llm (pyproject.toml) ... done
  Created wheel for tree-of-thoughts-llm: filename=tree_of_thoughts_llm-0.1.0-0.editable-py3-none-any.whl size=4927 sha256=15ef92b5fe0ef7d2c8a3105b7425db97eaa4388ae83073ab415b606947a83f8a
  Stored in directory: /tmp/pip-ephem-wheel-cache-68w8fegb/wheels/6e/

In [ ]:
import argparse
from tot.methods.bfs import solve
from tot.tasks.game24 import Game24Task

args = argparse.Namespace(backend='gpt-4', temperature=0.7, task='game24', naive_run=False, prompt_sample=None, method_generate='propose', method_evaluate='value', method_select='greedy', n_generate_sample=1, n_evaluate_sample=3, n_select_sample=5)

task = Game24Task()
ys, infos = solve(args, task, 900)
print(ys[0])

ModuleNotFoundError: ignored

In [ ]:
message=[{"role": "user", "content": gpt_prompt}]
response = openai.ChatCompletion.create(
    model="gpt-4",
    messages = message,
    temperature=0.2,
    max_tokens=1000,
    frequency_penalty=0.0
)
print(response)


APIRemovedInV1: ignored